# Data wrangling of Kickstarter project records

* Read bulk CSV files, select and process informative columns, drop duplicates, process dates and other data, check for missing data and outliers, and save to new cleaned CSVs

## Data wrangling procedure description

### 1. Raw data collection

The primary source of data for Kickstarter projects was obtained from online, monthly updated repositories collected by a web scraping company [Web Robots](https://webrobots.io/), with data available in JSON and CSV formats. For this project, I have downloaded 961 CSV files (43GB) available in March 2019, and saved them into the project directory /data/raw/kickstarter_csvs.

### 2. Inspection of the format and selection of the source data files

Inspection of the individual CSV files indicated that monthly records starting from March 2016 to present adhere to the same features (columns). These were selected for the capstone project analysis, while the older were discarded given the vast majority of the data adhered to the newer format and with reported >200K projects/samples it should contain enough data for statistical analysis and modeling. This selection also eliminated missing values in the dataset.

### 3. Inspection of the contents and selection of relevant features

Inspection of the columns indicate that many entries are in the form of JSON strings, with useful information embedded inside the directory structure. Some columns do not seem to contain information useful for this project (e.g. photo).
Dates seem to be stored in the form of a datastamp. Only informative relevant columns were selected, reducing their overall number from 32 to 18.

### 4. Concatenating data frames from multiple CSV files and elimination of duplicate entries

All 961 CSV files were read from the raw data directory tree and stored in a new concatenated dataframe.
The resulting dataset shows significant duplication of records as the same projects are recorded month after month. In fact around 90% of the entries are duplicates. While I have dropped identical records, there are still duplicate projects with the same id. As I am interested in comparing different projects and their final status, I eliminated all the ID duplicates and only keep the last one. Roughly 260K unique project records remained in the dataset, which corresponds to the number reported by Kicstarter.

### 5. Filtering and cleaning data into a useful format
As inspection of the entries showed, several were in the JSON string format. I have extracted the imporant 'category' feature from the appropriate JSON string in the 'category' column using 'map' method and JSON module. The resulting dataframe was saved in the interim data directory.

I loaded the interim CSV file into a new dataframe, and converted the date/time information from the datastamp format into a standard datetime format using a custom parser.

The 'category' feature seemed to be too fine grained with 169 different labels. Therefore I have processed this column to extract the more general category type descriptor which was contained as the first part of the category string (before '/'). The new cat_type colomn with 15 unique labels was added for future exploratory analysis.


### 6. Final check and saving the processed file

The check for erroneous outliers did not reveal any suspicious items, only very skewed distributions.
The resulting dataframe was saved in data/processed directory.
The original 43 GB has shrunk to ~90 MB CSV file.

## Data wrangling code

In [1]:
import os
import numpy as np
import pandas as pd
import json

* The data in the form of series of 961 CSV files (43GB) was downloaded from a webscraping company into a local directory.
* The cleaned data will be saved in a new CSV file.

In [2]:
base_path = '../data/raw/kickstarter_csvs'
interim_path = '../data/interim/kickstarter_csvs'
processed_path = '../data/processed'

* Inspection of the individual CSV files indicated that monthly records starting from March 2016 to present adhere to the same features (columns). These were selected for the capstone project analysis.
* This selection also eliminated missing values in the dataset.

In [3]:
df = pd.read_csv(os.path.join(base_path,'Kickstarter_2016-03-22/Kickstarter.csv'), parse_dates=['deadline'],infer_datetime_format=True)

In [4]:
df.head(1)

,id,photo,name,blurb,goal,pledged,state,slug,disable_communication,country,...,location,category,profile,spotlight,urls,source_url,friends,is_starred,is_backing,permissions
0,64486721,"{""small"":""https://ksr-ugc.imgix.net/projects/5...",Along The Lines Of...,Funds towards exhibiting photography at an Art...,300.0,300.0,successful,along-the-lines-of,False,GB,...,"{""country"":""FR"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""link_text_col...",True,"{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN


* Inspection of the columns indicate that many entries are in the form of JSON strings, with useful information embedded inside the directory structure. Some columns do not seem to contain information useful for this project (e.g. photo)
* Dates seem to be stored in the form of a datastamp.
* Only informative columns will be selected

In [5]:
# list of columns to select and save
properties = ['id', 'name', 'goal', 'pledged', 'usd_pledged', 'state', 'slug', 'country', 'currency', 'deadline', 'state_changed_at', 'created_at', 'launched_at', 'staff_pick', 'backers_count', 'blurb', 'spotlight','category']

* Read all 961 CSV files from the directory tree and save it as a new concatenated dataframe.
* During this conversion, extract the 'category' feature from the appropriate JSON string and add it as a new column.

In [6]:
new_dfn = [] # list of new dataframes for concatenation
for folderName, subfolders, filenames in os.walk(base_path):
    for filename in filenames:
        if filename.endswith('.csv'):
            csv_fname = os.path.join(folderName, filename)
            dfs = pd.read_csv(csv_fname)                     # read into DataFrame
            dfs['category'] = dfs['category'].map(lambda x: json.loads(x)['slug'])  # parse JSON
            dfs = dfs.reindex(columns=properties, copy=True)  # create a new dataframe
            new_dfn.append(dfs) #.assign(category=y.values))     # add a parsed category
            #y = dfs['category'].map(lambda x: json.loads(x)['slug'])  # parse JSON
            #dfn = dfs.reindex(columns=properties, copy=True)  # create a new dataframe
            #new_dfn.append(dfn.assign(category=y.values))     # add a parsed category

In [7]:
df_single = pd.concat(new_dfn, ignore_index=True)

In [8]:
df_single.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3935527 entries, 0 to 3935526
Data columns (total 18 columns):
id                  int64
name                object
goal                float64
pledged             float64
usd_pledged         float64
state               object
slug                object
country             object
currency            object
deadline            int64
state_changed_at    int64
created_at          int64
launched_at         int64
staff_pick          bool
backers_count       int64
blurb               object
spotlight           bool
category            object
dtypes: bool(2), float64(3), int64(6), object(7)
memory usage: 487.9+ MB


In [9]:
df_single.head(2)

,id,name,goal,pledged,usd_pledged,state,slug,country,currency,deadline,state_changed_at,created_at,launched_at,staff_pick,backers_count,blurb,spotlight,category
0,64486721,Along The Lines Of...,300.0,300.0,460.241994,successful,along-the-lines-of,GB,GBP,1368652795,1368652795,1365783741,1366060795,False,9,Funds towards exhibiting photography at an Art...,True,art/conceptual art
1,755137951,Portrait of #NOW,500.0,595.0,595.000000,successful,portrait-of-now,US,USD,1314486963,1314486965,1313619704,1313622963,False,5,This is Portrait of #NOW. I'm going to make a ...,True,art/conceptual art


* The dataset shows significant duplication of records as the same projects are recorded month after month.

In [10]:
df_single.duplicated().value_counts()

True     3609560
False     325967
dtype: int64

In [11]:
df_alldup = df_single.drop_duplicates()

In [12]:
df_alldup.shape

(325967, 18)

* While completely identical records were dropped, there are still duplicate projects with the same id. As I am interested in comparing different projects and their final status, I will eliminate all the ID duplicates and only keep the last one.

In [13]:
df_iddup = df_single.drop_duplicates(['id'], keep='last')

In [14]:
df_iddup.shape

(263765, 18)

* Roughly 260K unique project records were retained, which corresponds to the number reported by Kicstarter.

* Use the unique ID as index to be saved in a new CSV file

In [15]:
df_id = df_iddup.set_index('id').sort_index()

In [16]:
df_id.shape

(263765, 17)

In [17]:
df_id.to_csv(os.path.join(interim_path, 'kick_idn.csv'))        # save new dataframe

* The above is the processed single CSV file containing unique data over the history of kickstarter

* Convert the datastamp format into a standard datetime format using a custom parser and save in the data/processed directory

In [18]:
import datetime
# list of columns, which should be in datetime format
datecols = ['created_at', 'deadline', 'state_changed_at', 'launched_at']

In [19]:
# Function for converting from a timestamp to a datetime string
fdatpars = lambda x: datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S')

In [20]:
# Read-in a pre-processed CSV file, index based on project ID
df = pd.read_csv(os.path.join(interim_path, 'kick_idn.csv'), index_col='id', parse_dates=datecols, date_parser=fdatpars)

In [21]:
df.head(2)

,name,goal,pledged,usd_pledged,state,slug,country,currency,deadline,state_changed_at,created_at,launched_at,staff_pick,backers_count,blurb,spotlight,category
id,,,,,,,,,,,,,,,,,
18520,Grandma's are Life,15000.0,62.0,62.000000,failed,grandmas-are-life,US,USD,2016-11-18 09:32:40,2016-11-18 09:32:40,2016-10-19 08:13:18,2016-10-19 09:32:40,False,4,Raising money to help my grandmother recover f...,False,music/world music
21109,Meta,150.0,173.0,258.036032,successful,meta,GB,GBP,2015-05-06 17:00:00,2015-05-06 17:00:12,2015-04-07 10:07:05,2015-04-07 18:37:44,False,11,My work is performance based but I branch out ...,True,art/performance art


** Coarse-graining 'category'into 'cat_type' **

The 'category' feature seems to be too fine grained with 169 different labels. A more general category type will be added for future exploratory analysis.

In [22]:
# number of different categories
df['category'].value_counts().shape[0]

169

In [23]:
df['category'].value_counts().head(2)

design/product design    12477
games/tabletop games      9707
Name: category, dtype: int64

In [24]:
df['category'].value_counts().tail(2)

music/comedy        22
crafts/taxidermy    13
Name: category, dtype: int64

* Make coarse-grained category types based on the description before '/'

In [25]:
# make a list of general category types
cat_type = [x.split('/')[0] for x in df['category'].values]
print('Lengths of cat_type and category should be the same: ', len(cat_type), '<>', df['category'].shape[0])

Lengths of cat_type and category should be the same:  263765 <> 263765


In [26]:
df['cat_type'] = np.array(cat_type)

In [27]:
df['cat_type'].value_counts()

film & video    39441
music           36476
publishing      31807
technology      26736
games           25599
art             21090
design          16558
fashion         16024
food            15688
comics           8078
photography      6847
theater          6289
crafts           5743
journalism       4302
dance            3087
Name: cat_type, dtype: int64

In [28]:
len(df['cat_type'].unique())

15

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263765 entries, 18520 to 2147476221
Data columns (total 18 columns):
name                263764 non-null object
goal                263765 non-null float64
pledged             263765 non-null float64
usd_pledged         263765 non-null float64
state               263765 non-null object
slug                263765 non-null object
country             263765 non-null object
currency            263765 non-null object
deadline            263765 non-null datetime64[ns]
state_changed_at    263765 non-null datetime64[ns]
created_at          263765 non-null datetime64[ns]
launched_at         263765 non-null datetime64[ns]
staff_pick          263765 non-null bool
backers_count       263765 non-null int64
blurb               263750 non-null object
spotlight           263765 non-null bool
category            263765 non-null object
cat_type            263765 non-null object
dtypes: bool(2), datetime64[ns](4), float64(3), int64(1), object(8)
memory us

* Are there any suspicious irregularities or outliers in the data?

Look at the basic statistical descriptors

In [30]:
df.describe()

,goal,pledged,usd_pledged,backers_count
count,2.637650e+05,2.637650e+05,2.637650e+05,263765.000000
mean,4.671028e+04,1.196719e+04,1.085461e+04,128.543575
std,1.139242e+06,1.298566e+05,1.029759e+05,1046.538138
min,1.000000e-02,0.000000e+00,0.000000e+00,0.000000
25%,2.000000e+03,5.000000e+01,4.842330e+01,2.000000
50%,5.000000e+03,9.360000e+02,9.250000e+02,17.000000
75%,1.500000e+04,5.175000e+03,5.125000e+03,70.000000
max,1.000000e+08,2.902191e+07,2.033899e+07,219382.000000


In [31]:
df.sort_values('backers_count', ascending=False).head(2)

,name,goal,pledged,usd_pledged,state,slug,country,currency,deadline,state_changed_at,created_at,launched_at,staff_pick,backers_count,blurb,spotlight,category,cat_type
id,,,,,,,,,,,,,,,,,,
1955357092,Exploding Kittens,10000.0,8782571.99,8782571.99,successful,exploding-kittens,US,USD,2015-02-19 21:00:00,2015-02-19 21:01:19,2014-12-15 03:39:54,2015-01-20 13:00:19,True,219382,This is a card game for people who are into ki...,True,games/tabletop games,games
1386523707,Fidget Cube: A Vinyl Desk Toy,15000.0,6465690.30,6465690.30,successful,fidget-cube-a-vinyl-desk-toy,US,USD,2016-10-19 21:00:00,2016-10-19 21:00:00,2016-08-06 20:06:08,2016-08-30 16:02:09,True,154926,"An unusually addicting, high-quality desk toy ...",True,design/product design,design


* The distribution of funding and backers is strongly asymmetric, with a few very successful projects, but nothing suggests errors in the data

In [32]:
df.to_csv(os.path.join(processed_path, 'kick_idf.csv'))  

In [33]:
os.path.getsize(os.path.join(processed_path, 'kick_idf.csv'))/2**20

89.14581489562988

** Summary **

The original 43 GB has shrunk to ~90 MB CSV file. This was obtained by
* eliminating uninformative columns
* eliminating duplicate records
* extracting only useful information from JSON strings
* converting date and time from a stamp format into the standard form.
* adding a new 'cat_type' feature for grouping project category types